In [4]:
import numpy as np
import json

In [43]:
name = 'moreno_health'
layout = 'spherical'
filename = f'{name}-layout.json-{layout}.json'

## Load

In [44]:
with open(f'./result/{filename}') as file:
    data = json.load(file)

## Transformation Functions

In [45]:
def copy_positions(data):

    for node in data['nodes']:
        pos = np.array([node['pos3D'][0], node['pos3D'][2], node['pos3D'][1]])

        node['_position3D'] = {
            'x': pos[0],
            'y': pos[1],
            'z': pos[2]
        }

In [46]:
def normalize(data):

    for node in data['nodes']:
        pos = np.array([node['pos3D'][0], node['pos3D'][2], node['pos3D'][1]])
        pos *= 2
        pos += -1

        node['_position3D'] = {
            'x': pos[0],
            'y': pos[1],
            'z': pos[2]
        }

In [47]:
def centralize_groups(data):
    groups = np.unique(np.array([node['ancIdx'] for node in data['nodes'] if node['childIdx'] == []]))
    
    for group in groups:
        positions = [node['_position3D'] for node in data['nodes'] if node['ancIdx'] == group and node['childIdx'] == []]
        cx = np.mean([pos['x'] for pos in positions])
        maxx = np.max([pos['x'] for pos in positions])
        minx = np.min([pos['x'] for pos in positions])
        cz = np.mean([pos['z'] for pos in positions])
        maxz = np.max([pos['z'] for pos in positions])
        minz = np.min([pos['z'] for pos in positions])

        for position in positions:
            position['x'] -= cx
            position['x'] /= (maxx - minx)
            position['z'] -= cz + 0.15 # one meter additional offset
            position['z'] /= (maxz - minz)

These functions help create JSON graphs that can be deserialized in the IGVis project in Unity
It is important to copy 'pos3D' into '_position3D' otherwise all coordinates in Unity end up being 0
If you are processing data for the projected flat layout and want all groups to be centralized around 0,0 call centralize_groups in addition to normalize

In [48]:
if layout == 'spherical':
    # Call this for 'spherical' layouts
    copy_positions(data)
elif layout == 'flat':
    # Call this for 'flat' layouts
    normalize(data)
    centralize_groups(data)

## Save

In [49]:
with open(f'./result/{filename}', 'w') as file:
    json.dump(data, file)